In [4]:
using MAT
using PCHIPInterpolation
using DifferentialEquations

include("juliaIC.jl");

function loadmat(matfile)
    file = matopen(matfile)
    con = read(file, "ElSpecOut")
    close(file)
    return con
end


function getparams(con)
    ne = con["ne"] #size: (62, 674)
    
    iri = permutedims(con["iri"], (2, 1, 3))
    Tn_, Ti_, Te_, nN2, nO2, nO, nAr, nNOp, nO2p, nOp = [iri[i, :, :] for i in 1:size(iri)[1]]
    Tn = Tn_
    nNOp, nO2p, nOp = [i .* ne ./ sum([nNOp, nO2p, nOp]) for i in [nNOp, nO2p, nOp]]
    nion = cat(ne, nN2, nO2, nO, nAr, nNOp, nO2p, nOp; dims = 3)
    nion = permutedims(nion, (3, 1, 2))

    par = con["par"]
    Ti = par[:, 2, :]
    Te = par[:, 3, :]
    
    T = permutedims(cat(Te, Ti, Tn; dims = 3), (2, 1, 3)) #be aware of ordering!!!
    #size: (674, 62, 3)
    
    ts_ = dropdims(con["ts"]; dims = 2)
    te_ = dropdims(con["te"]; dims = 2)
    ts = ts_ .- ts_[1]
    te = te_ .- ts_[1]
    
    e_prod = permutedims(con["q"], (2, 1))
    #size 674x62
    
    h = dropdims(con["h"]; dims = 1)
    
    return ts, te, h, nion, T, e_prod
end


getparams (generic function with 1 method)

In [5]:
function interpolate_T(ts, T)
    i, j, k = size(T)
    T_intp = Array{Any}(undef, (j, k))

    for kk in 1:k
        for jj in 1:j
            T_intp[jj, kk] = Interpolator(ts, T[:, j, k])
        end
    end

    return T_intp
end

#function interpolate_q(ts, q)
#    i, k = size(q)
#    q_intp = Array{Any}(undef, k)
#
#    for kk in 1:k
#        q_intp[kk] = Interpolator(ts, q[:, k])
#    end
#
#    return T_intp
#end

interpolate_T (generic function with 1 method)

In [6]:
const path_reactions_file = "/Users/ost051/Documents/PhD/ElectronPrecipitation/Data/other/Reaction rates full set ext.txt"
const dndt, particles, reactions, ode_raw = juliaIC.initIC(path_reactions_file)
particles



nprodd[1](tt) .+ -1.0.* rr[1][4](tem).* nn[1, :].* nn[2, :] .+ 1.0.* rr[15][4](tem).* nn[12, :].* nn[14, :] .+ 1.0.* rr[18][4](tem).* nn[6, :].* nn[14, :] .+ 1.0.* rr[20][4](tem).* nn[10, :].* nn[5, :] .+ -1.0.* rr[21][4](tem).* nn[1, :].* nn[16, :] .+ 1.0.* rr[24][4](tem).* nn[17, :].* nn[14, :] .+ -1.0.* rr[27][4](tem).* nn[1, :].* nn[15, :] .+ -1.0.* rr[28][4](tem).* nn[1, :].* nn[8, :] .+ -1.0.* rr[29][4](tem).* nn[1, :].* nn[7, :] .+ 1.0.* rr[34][4](tem).* nn[11, :].* nn[14, :] .+ 1.0.* rr[44][4](tem).* nn[17, :].* nn[14, :] .+ 1.0.* rr[49][4](tem).* nn[10, :]

nprodd[2](tt) .+ -1.0.* rr[1][4](tem).* nn[1, :].* nn[2, :] .+ -1.0.* rr[2][4](tem).* nn[6, :].* nn[2, :] .+ -1.0.* rr[3][4](tem).* nn[9, :].* nn[2, :] .+ -1.0.* rr[4][4](tem).* nn[10, :].* nn[2, :]

nprodd[3](tt) .+ 1.2.* rr[1][4](tem).* nn[1, :].* nn[2, :] .+ 0.1.* rr[9][4](tem).* nn[7, :].* nn[14, :] .+ 0.7.* rr[23][4](tem).* nn[17, :].* nn[14, :] .+ 1.0.* rr[50][4](tem).* nn[4, :] .+ -1.0.* rr[52][4](tem).* nn[3, :]

np

19-element Vector{Vector{Any}}:
 [1, "O2+"]
 [2, "e-"]
 [3, "O(1D)"]
 [4, "O(1S)"]
 [5, "O"]
 [6, "N2+"]
 [7, "N(2D)"]
 [8, "N(4S)"]
 [9, "NO+"]
 [10, "O2+(a4P)"]
 [11, "O+(2D)"]
 [12, "O+(4S)"]
 [13, "O+(2P)"]
 [14, "O2"]
 [15, "NO"]
 [16, "N2"]
 [17, "N+"]
 [18, "H"]
 [19, "H+"]

In [7]:
function zerof(t)
    return 0
end

#make ion production part of ODE
function ion_production(qe, particles, n0)
    prod = Array{Any}(undef, length(particles))
    prod.= zerof
    i_O  = findall(p -> p[2] == "O", particles)[1]
    i_O2 = findall(p -> p[2] == "O2", particles)[1]
    i_N2 = findall(p -> p[2] == "N2", particles)[1]
    for i in 1:length(particles)
        if particles[i][2] == "e-"
            prod[i] = (t) -> qe(t, itp)
        end
        if particles[i][2] == "O+(4S)"
            prod[i] = (t) -> qe(t) .* n0[i_O , :]*0.56 ./(n0[i_O, :]*0.56 .+ n0[i_N2, :]*0.92 .+ n0[i_O2, :])
            #println(size(prod[i](0)))
        end
        if particles[i][2] == "O2+"
            prod[i] = (t) -> qe(t) .* n0[i_N2, :]*0.92 ./(n0[i_O, :]*0.56 .+ n0[i_N2, :]*0.92 .+ n0[i_O2, :])
        end
        if particles[i][2] == "N2+"
            prod[i] = (t) -> qe(t) .* n0[i_O2, :]      ./(n0[i_O, :]*0.56 .+ n0[i_N2, :]*0.92 .+ n0[i_O2, :])
        end
    end

    return Array{Any}(undef, length(particles))
end


ion_production (generic function with 1 method)

In [8]:
#function dummyf2(n, p, t)
#    dn = zeros(size(n))
#    reactions, nprod, dndt, T_intp, ih = p
#    @views TT = [t(0) for t in T_intp[ih, :]]
#    
#    for j in 1:size(n)
#        dn[j] = dndt[j](nprod, reactions, TT, n, t)[ih]
#    end 
#    return dn
#end


function dummyf2(n, p, t)
    dn = zeros(size(n))
    reactions, nprod, dndt, temp, ih = p
    temp_ = [t(0) for t in temp[1, :]]
    #temp_ = temp(t)
    #temp_2 = [temp_[i, :] for i in 1:size(temp_,1)]
    
    for j in 1:size(n)[1]
        #dn[j, :] .= nprod[j](t)
        #println(size(dndt[j](nprod, reactions, temp_2, n, t)))
        dn[j] = dndt[j](nprod, reactions, temp_, n, t)[ih]
    end 
    return dn
end


function IC_byHeight()
    path_reactions_file = "/Users/ost051/Documents/PhD/ElectronPrecipitation/Data/other/Reaction rates full set ext.txt"
    dndt, particles, reactions, ode_raw = juliaIC.initIC(path_reactions_file)

    con = loadmat("/Users/ost051/Documents/PhD/ElectronPrecipitation/log/testing/2023.05.12_09_56_23_mixf=0/ElSpec-iqt_IC_0.mat")
    ts, te, h, nion, T, e_prod = getparams(con)
    nh = length(h)
    np = length(particles)
    n0 = zeros(np, nh)
    #nion = [ne, nN2, nO2, nO, nAr, nNOp, nO2p, nOp]
    for i in 1:np
        if particles[i][2] == "N2"
            n0[i, :] = nion[2, :, 1]
            println(i)
        elseif particles[i][2] == "O2"
            n0[i, :] = nion[3, :, 1]
        elseif particles[i][2] == "O(1D)"
            n0[i, :] = nion[4, :, 1]
        elseif particles[i][2] == "NO+"
            n0[i, :] = nion[6, :, 1]
        elseif particles[i][2] == "O2+"
            n0[i, :] = nion[7, :, 1]
        elseif particles[i][2] == "O+(4S)"
            n0[i, :] = nion[8, :, 1]
        end
    end
    
    T_intp = interpolate_T(ts, T)
    println(size(T_intp))
    TT = Array{Float64}(undef, 3)
    
    #e_p_intp = interpolate_q(ts, e_prod)
    #println(size(e_p_intp))
    
    println(size(T))
    t_save = ts
    sol = Array{Any}(undef, nh)

    
    for ih in 1:nh
        #@time @views TT = [t(0) for t in T_intp[ih, :]]
        e_p_intp2 = Interpolator(ts, e_prod[:, ih])
        nprod = ion_production(e_p_intp2, particles, n0)
        #@time QQ = e_p_intp2(0)
        println("--")
        
        n0h = n0[:, ih]
        tspan = (ts, te)
        
                
        p = (reactions, nprod, dndt, T_intp[ih, :], ih)
        prob = dummyf2(n0h, p, 0)
        
        
        prob = ODEProblem(juliaIC.dummyf2, n0h, tspan, (reactions, nprod, dndt, T_intp[ih, :], ih))
        sol[i] = solve(prob, TRBDF2(autodiff=false), reltol = 1e-7, abstol = 1e-3, saveat = t_save)
    end
    
    
    return 0
end

#function setup()
#    
#end
#
#setup()
#

IC_byHeight()

"""
todo:
- map elspec ion densities to ionchem
- put some stuff in init, look up if state can be preserved in julia


"""

nprodd[1](tt) .+ -1.0.* rr[1][4](tem).* nn[1, :].* nn[2, :] .+ 1.0.* rr[15][4](tem).* nn[12, :].* nn[14, :] .+ 1.0.* rr[18][4](tem).* nn[6, :].* nn[14, :] .+ 1.0.* rr[20][4](tem).* nn[10, :].* nn[5, :] .+ -1.0.* rr[21][4](tem).* nn[1, :].* nn[16, :] .+ 1.0.* rr[24][4](tem).* nn[17, :].* nn[14, :] .+ -1.0.* rr[27][4](tem).* nn[1, :].* nn[15, :] .+ -1.0.* rr[28][4](tem).* nn[1, :].* nn[8, :] .+ -1.0.* rr[29][4](tem).* nn[1, :].* nn[7, :] .+ 1.0.* rr[34][4](tem).* nn[11, :].* nn[14, :] .+ 1.0.* rr[44][4](tem).* nn[17, :].* nn[14, :] .+ 1.0.* rr[49][4](tem).* nn[10, :]

nprodd[2](tt) .+ -1.0.* rr[1][4](tem).* nn[1, :].* nn[2, :] .+ -1.0.* rr[2][4](tem).* nn[6, :].* nn[2, :] .+ -1.0.* rr[3][4](tem).* nn[9, :].* nn[2, :] .+ -1.0.* rr[4][4](tem).* nn[10, :].* nn[2, :]

nprodd[3](tt) .+ 1.2.* rr[1][4](tem).* nn[1, :].* nn[2, :] .+ 0.1.* rr[9][4](tem).* nn[7, :].* nn[14, :] .+ 0.7.* rr[23][4](tem).* nn[17, :].* nn[14, :] .+ 1.0.* rr[50][4](tem).* nn[4, :] .+ -1.0.* rr[52][4](tem).* nn[3, :]

np

LoadError: UndefRefError: access to undefined reference

In [9]:
path_reactions_file = "/Users/ost051/Documents/PhD/ElectronPrecipitation/Data/other/Reaction rates full set ext.txt"
dndt, particles, reactions, ode_raw = juliaIC.initIC(path_reactions_file)

con = loadmat("/Users/ost051/Documents/PhD/ElectronPrecipitation/log/testing/2023.05.12_09_56_23_mixf=0/ElSpec-iqt_IC_0.mat")
ts, te, h, nion, T, e_prod = getparams(con)


nprodd[1](tt) .+ -1.0.* rr[1][4](tem).* nn[1, :].* nn[2, :] .+ 1.0.* rr[15][4](tem).* nn[12, :].* nn[14, :] .+ 1.0.* rr[18][4](tem).* nn[6, :].* nn[14, :] .+ 1.0.* rr[20][4](tem).* nn[10, :].* nn[5, :] .+ -1.0.* rr[21][4](tem).* nn[1, :].* nn[16, :] .+ 1.0.* rr[24][4](tem).* nn[17, :].* nn[14, :] .+ -1.0.* rr[27][4](tem).* nn[1, :].* nn[15, :] .+ -1.0.* rr[28][4](tem).* nn[1, :].* nn[8, :] .+ -1.0.* rr[29][4](tem).* nn[1, :].* nn[7, :] .+ 1.0.* rr[34][4](tem).* nn[11, :].* nn[14, :] .+ 1.0.* rr[44][4](tem).* nn[17, :].* nn[14, :] .+ 1.0.* rr[49][4](tem).* nn[10, :]

nprodd[2](tt) .+ -1.0.* rr[1][4](tem).* nn[1, :].* nn[2, :] .+ -1.0.* rr[2][4](tem).* nn[6, :].* nn[2, :] .+ -1.0.* rr[3][4](tem).* nn[9, :].* nn[2, :] .+ -1.0.* rr[4][4](tem).* nn[10, :].* nn[2, :]

nprodd[3](tt) .+ 1.2.* rr[1][4](tem).* nn[1, :].* nn[2, :] .+ 0.1.* rr[9][4](tem).* nn[7, :].* nn[14, :] .+ 0.7.* rr[23][4](tem).* nn[17, :].* nn[14, :] .+ 1.0.* rr[50][4](tem).* nn[4, :] .+ -1.0.* rr[52][4](tem).* nn[3, :]

np

([0.0, 0.4444444179534912, 0.8888888359069824, 1.3333332538604736, 1.777777910232544, 2.222222328186035, 2.6666667461395264, 3.1111111640930176, 3.5555996894836426, 4.000044107437134  …  295.1106083393097, 295.55512166023254, 295.99956607818604, 296.4440104961395, 296.888454914093, 297.3328993320465, 297.7773439884186, 298.22178840637207, 298.66623282432556, 299.11067724227905], [0.4444444179534912, 0.8888888359069824, 1.3333332538604736, 1.777777910232544, 2.222222328186035, 2.6666667461395264, 3.1111111640930176, 3.555555582046509, 4.000044107437134, 4.444488525390625  …  295.5550527572632, 295.99956607818604, 296.4440104961395, 296.888454914093, 297.3328993320465, 297.7773439884186, 298.22178840637207, 298.66623282432556, 299.11067724227905, 299.55512166023254], [95.85472385504806, 96.73278239417829, 97.61084093330851, 98.48889947243873, 99.36695801156895, 100.24501655069918, 101.1230750898294, 102.00113362895962, 102.87919216808984, 103.75725070722007  …  141.51376788981963, 142.39

In [11]:
println(size(nion))

T_intp = interpolate_T(ts, T)
size(T_intp)
[t(0) for t in T_intp[1, :]]

nh = length(h)
np = length(particles)
n0 = zeros(np, nh)
    
#nion = [ne, nN2, nO2, nO, nAr, nNOp, nO2p, nOp]
for i in 1:np
    if particles[i][2] == "N2"
        n0[i, :] = nion[2, :, 1]
        println(i)
    elseif particles[i][2] == "O2"
        n0[i, :] = nion[3, :, 1]
    elseif particles[i][2] == "O(1D)"
        n0[i, :] = nion[4, :, 1]
    elseif particles[i][2] == "NO+"
        n0[i, :] = nion[6, :, 1]
    elseif particles[i][2] == "O2+"
        n0[i, :] = nion[7, :, 1]
    elseif particles[i][2] == "O+(4S)"
        n0[i, :] = nion[8, :, 1]
    end
end
    
    T_intp = interpolate_T(ts, T)
    println(size(T_intp))
    TT = Array{Float64}(undef, 3)
    
    #e_p_intp = interpolate_q(ts, e_prod)
    #println(size(e_p_intp))
    
    println(size(T))
    t_save = ts
    sol = Array{Any}(undef, nh)

(8, 62, 674)
16
(62, 3)
(674, 62, 3)


62-element Vector{Any}:
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
   ⋮
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef

In [ ]:
np = length(particles)
nh = length(h)
n0 = zeros(np, nh)
    #nion = [ne, nN2, nO2, nO, nAr, nNOp, nO2p, nOp]
    for i in 1:np
        if particles[i][2] == "N2"
            n0[i, :] = nion[2, :, 1]
            println(i)
        elseif particles[i][2] == "O2"
            n0[i, :] = nion[3, :, 1]
        elseif particles[i][2] == "O(1D)"
            n0[i, :] = nion[4, :, 1]
        elseif particles[i][2] == "NO+"
            n0[i, :] = nion[6, :, 1]
        elseif particles[i][2] == "O2+"
            n0[i, :] = nion[7, :, 1]
        elseif particles[i][2] == "O+(4S)"
            n0[i, :] = nion[8, :, 1]
        end
    end

n0

16


19×62 Matrix{Float64}:
 1.80178e10  2.80073e10  3.97494e10  …  7.22771e10  7.24361e10  7.23335e10
 0.0         0.0         0.0            0.0         0.0         0.0
 3.00082e17  3.08221e17  3.11227e17     1.10853e16  1.07078e16  1.03489e16
 0.0         0.0         0.0            0.0         0.0         0.0
 0.0         0.0         0.0            0.0         0.0         0.0
 0.0         0.0         0.0         …  0.0         0.0         0.0
 0.0         0.0         0.0            0.0         0.0         0.0
 0.0         0.0         0.0            0.0         0.0         0.0
 9.0089e9    1.40036e10  1.98747e10     3.61385e10  3.6218e10   3.61668e10
 0.0         0.0         0.0            0.0         0.0         0.0
 0.0         0.0         0.0         …  0.0         0.0         0.0
 3.00297e9   4.66788e9   6.6249e9       1.20462e10  1.20727e10  1.20556e10
 0.0         0.0         0.0            0.0         0.0         0.0
 3.96627e18  3.39376e18  2.8931e18      3.31351e15  3.12798e15  2

In [ ]:
"""
Performance:
1. Avoid containers with abstract types
    => define arrays as float64 explicitly


1. Type-stable code => no abstract types => define types explicitly as eg. Float64 of Int32
2. 
"""

"Performance:\n1. Avoid containers with abstract types\n    => define arrays as float64 explicitly\n\n\n1. Type-stable code => no abstract types => define types explicitly as eg. Float64 of Int32\n2. \n"

In [10]:
juliaIC.dndt_str

LoadError: UndefVarError: `dndt_str` not defined